In [1]:
from functions import *
from random import sample
from htmldate import find_date
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import json
from langid.langid import LanguageIdentifier, model
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib

In [6]:
# Number of Webpages
base_path = "/media/ruben/Data Drive/react-data/protest/selection2"

dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    dt.update({photo:0})

    folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
    print(photo, folders)
    
    for fol in folders:
            htmls = [f for f in os.listdir(os.path.join(photo_folder,fol,"html")) if ".html" in f]
            dt[photo] += len(htmls)

d = pd.DataFrame(dt.items(),columns = ['image','n_webpages'])
d = d.sort_values('image',ascending=False)

1 ['1_1', '1_2']
10 ['10_1', '10_2']
12 ['12_1', '12_2', '12_3', '12_4', '12_5']
13 ['13_1', '13_2', '13_3']
14 ['14_1']
15 ['15_1', '15_2']
2 ['2_1', '2_2']
24 ['24_1', '24_2']
29 ['29_1', '29_2', '29_3', '29_4']
30 ['30_1', '30_2']
5 ['5_1', '5_2']
7 ['7_1', '7_2']
8 ['8_1', '8_2', '8_3', '8_4']
9 ['9_1', '9_2', '9_3']
Athens ['Athens_1', 'Athens_2']
Barcelona ['Barcelona_1', 'Barcelona_2']
Berlin ['Berlin_1', 'Berlin_2']
Copenhagen ['Copenhagen_1', 'Copenhagen_2']
HalifaxNS ['HalifaxNS_1', 'HalifaxNS_2']
London ['London_1', 'London_2']
London4 ['London4_1', 'London4_2']
Rafah ['Rafah_1', 'Rafah_2', 'Rafah_3', 'Rafah_4', 'Rafah_5']
Rome2 ['Rome2_1', 'Rome2_2', 'Rome2_3', 'Rome2_4', 'Rome2_5']
Rome3 ['Rome3_1', 'Rome3_2']
SanFranciscoCA ['SanFranciscoCA_1', 'SanFranciscoCA_2']
SouthAfrica ['SouthAfrica_1', 'SouthAfrica_2']


In [7]:
# Number of Scrape-Images
dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    dt.update({photo:0})

    folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]

    for fol in folders[:-1]:
        htmls = len([f for f in os.listdir(os.path.join(photo_folder,fol,"img")) if ".txt" not in f])
        if htmls == 0:
            dt[photo] += 1
        else:
            dt[photo] += htmls
            
d2 = pd.DataFrame(dt.items(),columns = ['image','n_scrape-images'])
d2 = d2.sort_values('image',ascending=False)
d['n_scrape-images'] = d2['n_scrape-images'] + 1
d['scrape-ratio'] = 1 - round(d['n_scrape-images'] / d['n_webpages'],2)
d = d.sort_values('image',ascending=False) 

In [8]:
# Number of Text-Scraped Pages
dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
        photo_folder = os.path.join(base_path, photo)
        dt.update({photo:0})
        
        folders = [fol for fol in os.listdir(photo_folder) if os.path.isdir(os.path.join(photo_folder,fol)) and "source" not in fol and "context" not in fol]
        
        for fol in folders:
            text_jsons = [f for f in os.listdir(os.path.join(photo_folder,fol,"txt")) if ".json" in f]
            
            for js in text_jsons:
                fn = os.path.join(photo_folder,fol,"txt",js)
                with open(fn,'r') as fp:
                    js = json.load(fp)
                    list_sentences = [v for k,v in js.items()]
                    list_sentences = [len(item) for sublist in list_sentences for item in sublist]
                    dt[photo] += sum(list_sentences)
                    
d3 = pd.DataFrame(dt.items(),columns = ['image','n_scraped-text-characters'])
d3 = d3.sort_values('image',ascending=False)
d['n_scraped-text-characters'] = d3['n_scraped-text-characters']
d['char-page-ratio'] = round(d['n_scraped-text-characters'] / d['n_webpages'],2)

In [9]:
# Number of Context-Images

dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
        photo_folder = os.path.join(base_path, photo)
        dt.update({photo:0})
        
        cimgs = [f for f in os.listdir(os.path.join(photo_folder,"context_images")) if ".txt" not in f]
        dt[photo] += len(cimgs)
        
d4 = pd.DataFrame(dt.items(),columns = ['image','n_context-images'])
d4 = d4.sort_values('image',ascending=False)
d['n_context-images'] = d4['n_context-images']

In [10]:
# Number of Dates

dt = dict()
for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
        photo_folder = os.path.join(base_path, photo)
        dt.update({photo:0})
        
        with open(os.path.join(base_path,photo,"dates.txt"), 'r') as f:
            dates = f.readlines()
            dates = [d for d in dates if d.split('|')[1].replace('\n','') != "na" and "ERROR" not in d.split('|')[1]]
            dt[photo] += len(dates)
d5 = pd.DataFrame(dt.items(),columns = ['image','n_dates'])
d5 = d5.sort_values('image',ascending=False)
d['n_dates'] = d5['n_dates']

In [11]:
# Export

d.to_csv('descriptive-full.csv',index=False)

In [174]:
# Languages

dt = pd.DataFrame()

for photo in [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]:
    photo_folder = os.path.join(base_path, photo)
    
    with open(os.path.join(photo_folder,'languages-'+photo+".json"),'r') as f:
        lang = json.load(f)
        lang = lang['1']
        lang = dict(Counter([v[0] for k,v in lang.items()]))
        lang = pd.DataFrame(list(lang.items()))
        lang['photo'] = photo
    dt = dt.append(lang)
dt.columns = ['lang','count','photo']
d6 = dt.pivot(index="photo",columns="lang",values="count").reset_index().fillna(0)